In [8]:
#| hide
from police_risk_open_ai.llm import *
from police_risk_open_ai.crawl import *
import os
from dotenv import load_dotenv
import pandas as pd
import openai
from openai.embeddings_utils import distances_from_embeddings

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

df = pd.read_parquet('processed/embeddings.parquet')

# CopBot - Teaching Policing to the AI

In case you've been living under a very large rock, the entire world has been getting very, very excited about ChatGPT. It does a fantastic job of digestign large bits of information, and using that to answer questions...*mostly well*.

Digesting large volumes of text is pretty key policing challenge: you'll have thousand pages worth of reports, or statements, and you'll have to find the proverbial "needle in the haystack"... so I figured I'd start there, and see if I could tweak a model to help digest policing risk.

![CopBot](images/cop_bot.png)

So can an AI learn to speak police, and make vaguely convicing risk assessments? Apparently so! I scraped a bunch of policing guidance, passed it to OpenAI, and asked it to predict risk for missing people.  The [project code is available here](https://github.com/AndreasThinks/police-risk-open-ai), or if you want to skip straight to the fun part, you [can try out the prototype instead.](https://andreasthinks-police-risk-open-ai-main-y0l65v.streamlit.app/) 

## Learning Cop Speak

So how do you teach a robot to assess risk?  Well, you go to the official [College of Policing authorised professional practice](https://www.college.police.uk/app).

The next stage is embeddings, which essentially converts your text into a numerical representation of how your model believes the terms to be associated. As usual, if you want to learn this stuff, I really recommend the [Fast.ai course.](https://course.fast.ai/)

One of the most quickly impactive tool is semantic similarity, allowing us to identify documents of note... and that's the key. We take our documetns, and essentially feed them to a CHatGPT equivalent.

Because I've used [nbdev](https://nbdev.fast.ai/), there is some wonderful documentation to bwose if you want to read my source code. 

In [3]:
answer_question(df, question="What day is it?")

"I don't know."

Interestingly, it's more constrained than chat gpt...but it does knowa bout missing people!

In [4]:
answer_question(df, question="What are the most important factors to consider when searching for a missing person?", debug=True)

Context:
Very detailed information and a lifestyle profile will be needed in high-risk cases consider taking a full statement from the person reporting the missing person as well as any other key individuals (for example, the last person to see them) conduct initial searches of relevant premises, the extent and nature of the search should be recorded (see Search) consider seizing electronic devices, computers, and other documentation, (for example, diaries, financial records and notes) and obtain details of usernames and passwords obtain photos of the missing person; these should ideally be current likeness of the missing person and obtained in a digital format obtain details of the individual’s mobile phone and if they have it with them; if the missing person has a mobile phone arrange for a TextSafe© to be sent by the charity, Missing People obtain details of any vehicles that they may have access to and place markers on relevant vehicles on the PNC without delay consider obtaining a

'The primary consideration for the first responder is the safety of the missing person. Judgements made at this early stage may have a significant impact on the outcome of the investigation. The initial investigating officer should begin the investigation by identifying places where the person might be, check information and assumptions, corroborate what they have been told, review the risk assessment, seek and secure evidence, conduct appropriate searches, conduct appropriate intelligence checks, continually reassess the level of risk using the risk principles, assess the level of support required for the missing person’s family, residential worker or foster carer as appropriate, consider seizing electronic devices, computers, and other documentation, obtain photos of the missing person, obtain details of the individual’s mobile phone, obtain'

In [5]:
about_yannik = """ Yannik is a 15 year old boy. He has recently been down, and was reported missing by his parents as he did not return home from school today.

His friends are worried he may be depressed, and when he apparently told one a few days ago 'if it doesn't get any better, I'm going to end it soon'
"""

yannik_answer = machine_risk_assessment(about_yannik, df, debug=True)
yannik_answer

Context:
 First published 22 November 2016  Updated 15 March 2023   Latest changes  Written by College of Policing  Missing persons  30 mins read   Implications for the UK leaving the European Union are currently under review – please see APP on international investigation for latest available detail on specific areas, for example: Schengen Information System Europol INTERPOL Joint Investigation Teams This section provides additional information to aid the investigation based on the vulnerability of the individual and the circumstances in which they are missing. Missing children Safeguarding young and vulnerable people is a responsibility of the police service and partner agencies (see Children Act 2004). When the police are notified that a child is missing, there is a clear responsibility on them to prevent the child from coming to harm. Where appropriate, a strategy meeting may be held. For further information see: Voice of the child  Voice of the child practice briefing  Section 11 

('Graded as High risk, because of the below risk factors: \n- Yannik is a 15 year old boy who has been reported missing by his parents\n- His friends are worried he may be depressed\n- He has expressed suicidal ideation to one of his friends',
 " First published 22 November 2016  Updated 15 March 2023   Latest changes  Written by College of Policing  Missing persons  30 mins read   Implications for the UK leaving the European Union are currently under review – please see\xa0APP\xa0on international investigation\xa0for latest available detail on specific areas, for example: Schengen Information System Europol INTERPOL Joint Investigation Teams This section provides additional information to aid the investigation based on the vulnerability of the individual and the circumstances in which they are missing. Missing children Safeguarding young and vulnerable people is a responsibility of the police service and partner agencies (see\xa0Children Act 2004). When the police are notified that a 

Let's answer a question from here
https://www.how2become.com/blog/police-sergeants-inspectors-exam/


In [6]:
question = """ Officer Jennings is on his evening patrol. He is just about to finish for the day. As he walks down the street, he is approached by a man named Mark, who claims that he saw a man (named Steven) driving down a road not far from the location. Mark claims that he saw Steven drive into a cyclist, before driving off without stopping. Luckily, the cyclist was unharmed. The cyclist was named Kevin. Mark spoke to Kevin, and discovered that he is a 42 year old man, with a wife and two daughters.

Fifteen minutes later, Officer Jennings manages to stop the car being driven by Steven. He pulls him over to the side of the road, and orders him to step out of the car.

 

Referring s.6 (5) of the Road Traffic Act 1988, is Officer Jennings within his legal rights to order that Steven takes a preliminary breath test?

A – No. Officer Jennings has no right to tell Steven what he can and can’t do. He should never have stopped Steven in the first place.

B – No. In order for Officer Jennings to do this, an accident must have happened. The fact that Officer Jennings suspects an accident has taken place, does not meet this requirement.

C – Yes. However, the breath test must take place within or close to an area where the requirements for Steven to cooperate, can be imposed.

D – Yes. Officer Jennings can tell Steven to do whatever he wants, as he’s a police officer."""

answer_sergeant_exam_question(df,question)

'B - No. In order for Officer Jennings to do this, an accident must have happened. The fact that Officer Jennings suspects an accident has taken place, does not meet this requirement.'

Sadly, the correct answer is C.... so no luck.

In [7]:
question = """Sarah is walking to work one morning, when she is approached from behind by Henry and Jacob.

‘We won’t hurt you, as long as you give us the bag,’ Henry says.

‘You’re not getting it!’ Sarah shouts.

Henry grabs Sarah and holds a knife to her throat, whilst Jacob tries to snatch her bag.

Sarah fights with her attackers, and begins to run away. As the two men chase her, she trips and bangs her head on the pavement. She is taken to hospital and dies from head trauma.

Based on the above information, which of the following options is correct?

A – Jacob cannot be held accountable for the death of Sarah, as he simply tried to take her bag.

B – Jacob and Henry will be charged with attempted robbery, but not in the death of Sarah.

C – Jacob and Henry could be considered liable for the death of Sarah.

D – Sarah’s death cannot be blamed on Henry and Jacob, as it was her choice to run away."""

answer_sergeant_exam_question(df,question)

"C - Jacob and Henry could be considered liable for the death of Sarah because they were both involved in the attempted robbery and Sarah's death was a result of their actions."

That's one point to CopBot!